<a href="https://colab.research.google.com/github/maozida880/programming-specification/blob/main/HIVE_SQL%E5%BC%80%E5%8F%91%E8%A7%84%E8%8C%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **捷停车 HIVE SQL开发规范**

---

**引言：**

欢迎来到数据世界！Hive是我们在大数据领域处理和分析海量数据的重要工具。养成良好的SQL编写习惯，不仅能让你的代码逻辑清晰、易于维护，更能极大提升查询效率，节约宝贵的计算资源。

本规范旨在为你提供一套清晰、可行、详细的HIVE SQL开发指导，通过丰富的“正反”案例，帮助你快速上手，并培养出卓越的编程习惯。请仔细阅读，并在日常开发中严格遵守。

---

### **一、 命名规范 (Naming Conventions)**

清晰的命名是代码自解释性的第一步。

#### **1.1 表和字段 (Tables and Columns)**

* **规则:**
    * 全部使用小写字母。
    * 单词之间使用下划线 `_` 分隔。
    * 表名应清晰体现其业务含义、数据来源和更新频率（如适用）。推荐结构：`[业务域]_[业务主题]_[数据粒度]_[分区信息]`。
    * 字段名应准确描述其内容，避免使用无意义的缩写。
    * 禁止使用SQL关键字作为表名或字段名 (如 `select`, `from`, `where` 等)。

* **示例:**

| 规范类型 | 👍 正确示例 (Good) | 👎 错误示例 (Bad) |
| :--- | :--- | :--- |
| **表名** | `dwd_user_behavior_log_di` (数据仓库明细层-用户行为日志-天分区) | `UserBehavior`, `tbl_user_log`, `log` |
| **字段名** | `user_id`, `order_amount`, `session_start_time` | `userid`, `orderAmount`, `time` |

#### **1.2 临时表/视图 (Temporary Tables / Views)**

* **规则:** 临时表或中间查询结果的CTE (Common Table Expression) 命名应以 `tmp_` 或 `with_` 作为前缀，并简要说明其用途。

* **示例:**

In [ ]:
-- 👍 正确示例 (Good)
WITH tmp_user_paid_orders AS (
    SELECT user_id, order_id
    FROM dws_trade_user_order_1d
    WHERE dt = '2025-07-30' AND order_status = 'PAID'
)
SELECT ...

In [ ]:
-- 👎 错误示例 (Bad)
WITH t1 AS (
    SELECT user_id, order_id
    FROM dws_trade_user_order_1d
    WHERE dt = '2025-07-30' AND order_status = 'PAID'
)
SELECT ...

---

### **二、 代码风格与格式 (Code Style and Formatting)**

统一的风格让代码更易读、更专业。

#### **2.1 缩进与换行 (Indentation and Line Breaks)**

* **规则:**
    * 使用2个或4个空格进行缩进，不要使用Tab键。
    * `SELECT`, `FROM`, `WHERE`, `GROUP BY`, `HAVING`, `ORDER BY`, `JOIN` 等主关键字独占一行并左对齐。
    * `SELECT` 后的字段，每个字段独占一行并对齐。
    * `JOIN` 条件和 `WHERE` 条件，每个条件独占一行，并使用 `AND/OR` 开头。
    * 复杂的逻辑表达式应使用括号 `()` 来明确优先级。

* **示例:**

In [ ]:
-- 👍 正确示例 (Good)
SELECT
    a.user_id,
    a.user_name,
    b.order_count
FROM
    dwd_user_info_df a
JOIN
    (
        SELECT
            user_id,
            COUNT(order_id) AS order_count
        FROM
            dwd_order_info_di
        WHERE
            dt = '2025-07-30'
            AND order_status = 'COMPLETED'
        GROUP BY
            user_id
    ) b
ON
    a.user_id = b.user_id
WHERE
    a.register_date >= '2025-01-01'
    AND b.order_count > 5;

In [ ]:
-- 👎 错误示例 (Bad)
select a.user_id, a.user_name, b.order_count from dwd_user_info_df a join (select user_id, count(order_id) as order_count from dwd_order_info_di where dt='2025-07-30' and order_status='COMPLETED' group by user_id) b on a.user_id = b.user_id where a.register_date>='2025-01-01' and b.order_count>5;

#### **2.2 大小写 (Capitalization)**

* **规则:**
    * 所有SQL关键字大写 (e.g., `SELECT`, `FROM`, `WHERE`, `JOIN`, `AS`)。
    * 所有函数名大写 (e.g., `COUNT()`, `SUM()`, `CAST()`, `COALESCE()`)。
    * 表名、字段名和别名小写。

* **示例:**

In [ ]:
-- 👍 正确示例 (Good)
SELECT
    user_id,
    UPPER(user_name) AS user_name_upper,
    COUNT(DISTINCT order_id) AS distinct_order_count
FROM
    ods_order_log
WHERE
    dt = '2025-07-30';

In [ ]:
-- 👎 错误示例 (Bad)
select
    userid,
    upper(username) as UserNameUpper,
    count(distinct OrderID) as distinct_order_count
from
    ods_order_log
where
    dt = '2025-07-30';

---

### **三、 注释规范 (Commenting)**

好的注释是代码的“说明书”。

* **规则:**
    * 每个SQL脚本的开头，必须包含作者、创建日期、修改历史和脚本功能的说明。
    * 对于复杂的业务逻辑、关键的计算步骤或临时的“魔法数字”，必须在其上方或右方添加注释。
    * 单行注释使用 `--`，多行注释使用 `/* ... */`。

* **示例:**

In [ ]:
/*
 * @Author: your_name
 * @Date: 2025-08-01
 * @Description: 计算每日活跃且付费的用户列表及其订单总额。
 * @Modification History:
 * 2025-08-02, your_name, 修正了订单金额为0的过滤条件。
*/

-- 临时表：筛选出昨日的付费用户
WITH tmp_paid_users AS (
    SELECT
        user_id
    FROM
        dwd_payment_log_di
    WHERE
        dt = '2025-07-31'
        AND payment_amount > 0 -- 过滤掉金额为0的异常支付记录
)

SELECT
    dau.user_id,
    SUM(o.order_amount) AS total_order_amount
FROM
    dws_user_dau_1d dau -- Daily Active Users
JOIN
    dwd_order_info_di o ON dau.user_id = o.user_id AND o.dt = '2025-07-31'
JOIN
    tmp_paid_users pu ON dau.user_id = pu.user_id
WHERE
    dau.dt = '2025-07-31'
GROUP BY
    dau.user_id;

---

### **四、 性能与优化 (Performance and Optimization)**

这是HIVE SQL开发的核心，直接影响成本和效率。

#### **4.1 数据过滤与分区裁剪 (Data Filtering and Partition Pruning)**

* **核心思想:** **尽早、尽可能地过滤数据。**
* **规则:**
    * `WHERE` 子句中必须包含分区字段的过滤条件，且分区字段过滤条件应写在最前面。这是HIVE性能优化的第一生命线。
    * 对于多层分区的表，尽量提供所有层级的分区条件。
    * 避免在分区字段上使用函数，这会导致分区裁剪失效。

* **示例:**

In [ ]:
-- 👍 正确示例 (Good)
SELECT
    user_id,
    page_id
FROM
    dwd_page_view_log_hi
WHERE
    dt = '2025-07-30' -- 分区裁剪生效
    AND hour = '10'
    AND event_type = 'click';

In [ ]:
-- 👎 错误示例 (Bad) - 分区裁剪失效
SELECT
    user_id,
    page_id
FROM
    dwd_page_view_log_hi
WHERE
    -- 在分区字段上使用函数，将导致全表扫描！
    SUBSTR(dt, 1, 7) = '2025-07'
    AND event_type = 'click';

**改进方法：**

In [ ]:
-- 对上一个错误示例的正确改写
SELECT
    user_id,
    page_id
FROM
    dwd_page_view_log_hi
WHERE
    dt >= '2025-07-01' AND dt <= '2025-07-31' -- 使用范围查询
    AND event_type = 'click';

#### **4.2 JOIN 操作优化**

* **核心思想:** **用小表驱动大表，并善用JOIN策略。**
* **规则:**
    * **`JOIN` 顺序:** 在`FROM`子句中，将小表（经过`WHERE`过滤后行数最少的表）放在`JOIN`的左边。虽然Hive优化器会尝试重排，但这是一个好习惯。
    * **`LEFT JOIN` vs `INNER JOIN`:** 如果业务逻辑允许，优先使用`INNER JOIN`，因为它可以过滤掉更多不匹配的数据。
    * **`MapJoin`:** 对于一个大表和一个或多个小表的`JOIN`，使用`MapJoin`可以极大地提升性能。小表会被加载到内存中，避免了Reduce阶段的数据shuffle。通常可以设置`set hive.auto.convert.join=true;`让Hive自动转换，但有时需要手动指定。
    * **`JOIN`条件的字段类型:** 确保`JOIN`两边关联的字段类型完全一致，避免隐式类型转换带来的性能损耗。
    * **避免在`ON`子句中进行复杂计算:** `ON`子句中的计算（如函数、case when）会降低效率，应尽量提前在子查询中处理好。

* **示例 (MapJoin):**

In [ ]:
-- 👍 正确示例 (Good) - 手动指定MapJoin
SELECT /*+ MAPJOIN(b) */
    a.order_id,
    a.order_amount,
    b.user_name
FROM
    dwd_order_info_di a
JOIN
    dwd_user_info_df b ON a.user_id = b.user_id -- 假设b是小表
WHERE
    a.dt = '2025-07-30';

* **示例 (ON子句):**

In [ ]:
-- 👎 错误示例 (Bad)
SELECT *
FROM order_table o
JOIN user_table u
  ON SUBSTR(o.order_time, 1, 10) = u.register_date; -- ON子句中存在函数

In [ ]:
-- 👍 正确示例 (Good)
WITH tmp_order AS (
    SELECT
        *,
        SUBSTR(order_time, 1, 10) AS order_date
    FROM
        order_table
)
SELECT *
FROM tmp_order o
JOIN user_table u
  ON o.order_date = u.register_date;

#### **4.3 `GROUP BY` 与 `COUNT(DISTINCT)` 优化**

* **核心思想:** **减少数据倾斜，避免单点瓶颈。**
* **规则:**
    * **数据倾斜:** 如果`GROUP BY`的某个key值过多，会导致数据倾斜。可以尝试开启Hive的倾斜优化`set hive.groupby.skewindata=true;`，它会通过两个MapReduce Job来处理。
    * **`COUNT(DISTINCT)`:** 单个`COUNT(DISTINCT)`会使用一个Reducer，效率低下。当需要对多个字段进行去重计数时，应使用多个`GROUP BY`子查询再合并，或者使用`SUM(CASE WHEN ...)`的技巧。

* **示例 (`COUNT(DISTINCT)`):**

In [ ]:
-- 👎 错误示例 (Bad) - 效率低下
SELECT
    dt,
    COUNT(DISTINCT user_id),
    COUNT(DISTINCT session_id)
FROM
    dwd_page_view_log_di
WHERE
    dt = '2025-07-30'
GROUP BY
    dt;

In [ ]:
-- 👍 正确示例 (Good) - 转换为子查询/UNION ALL
SELECT dt, COUNT(user_id), COUNT(session_id) FROM (
    SELECT dt, user_id, null as session_id FROM dwd_page_view_log_di WHERE dt = '2025-07-30' GROUP BY dt, user_id
    UNION ALL
    SELECT dt, null as user_id, session_id FROM dwd_page_view_log_di WHERE dt = '2025-07-30' GROUP BY dt, session_id
) tmp GROUP BY dt;

#### **4.4 `ORDER BY` vs `SORT BY` vs `DISTRIBUTE BY` vs `CLUSTER BY`**

* **`ORDER BY`:** 全局排序，所有数据进入一个Reducer，当数据量大时，**极易造成性能瓶颈甚至OOM**。**请谨慎使用！**
* **`SORT BY`:** 局部排序，只保证在每个Reducer内部有序。通常和`DISTRIBUTE BY`联用。
* **`DISTRIBUTE BY`:** 控制数据如何分区到Reducer。根据指定的字段哈希值将数据分发到不同的Reducer。
* **`CLUSTER BY`:** `DISTRIBUTE BY` 和 `SORT BY` 的结合，但排序是升序的。

* **规则:**
    * 如果只是想取Top N，应使用`ORDER BY ... LIMIT N`，Hive会对此进行优化，或者使用窗口函数。
    * 如果需要对最终输出文件进行排序，但不需要全局严格有序，使用`SORT BY`。
    * 当需要控制数据分发并进行局部排序时，使用`DISTRIBUTE BY ... SORT BY`。

* **示例:**

In [ ]:
-- 需求：找出每个城市销售额最高的10个用户
-- 👍 正确示例 (Good) - 使用窗口函数，避免全局ORDER BY
SELECT
    city,
    user_id,
    total_amount
FROM
    (
        SELECT
            city,
            user_id,
            total_amount,
            ROW_NUMBER() OVER(PARTITION BY city ORDER BY total_amount DESC) AS rn
        FROM
            dws_user_consumption_1d
        WHERE
            dt = '2025-07-30'
    ) t
WHERE
    rn <= 10;

In [ ]:
-- 👎 错误示例 (Bad) - 极可能导致失败
SELECT * FROM huge_table ORDER BY column_a;

#### **4.5 其他建议**

* **`SELECT *` 禁令:** **严禁在生产环境的SQL中使用 `SELECT *`**。明确写出需要的字段，可以减少数据读取量，并且在表结构变更时代码依然健壮。
* **使用`UNION ALL`代替`UNION`:** `UNION`会去重，涉及到全局排序和比较，非常耗时。如果确认数据没有重复，或允许重复，务必使用`UNION ALL`。
* **合理使用数据类型:**
    * 用`BIGINT`存储用户ID等长整型，而不是`STRING`。
    * 用`DECIMAL`存储金额，避免`DOUBLE`或`FLOAT`的精度问题。
    * 日期使用标准的`DATE`或`TIMESTAMP`类型。
* **利用向量化查询:** 设置`set hive.vectorized.execution.enabled = true;`可以一次性处理1024行数据，大幅提升扫描、聚合、过滤和join的性能。